# Decision Trees Alogorithms and Impurity Measures: ID3, CART, and C4.5; Entropy, Gini Index, and Classification Error

Gini Impurity and Entropy are so similar that they are nearly the same thing, but they do have some distinctions. They come out of different fields of science. They can be used interchangably in many cases, but for a small percentage of use cases it is important to know which to choose.

Making this choice comes down to the nature of your data.

Since neither method results in a more accurate tree, the common consensus is that Gini is best to use since it is computationally less expensive.

## Gini Impurity is for continuous attributes

and is used to minimize misclassification.

## Entropy is for attributes that occur in classes

and is used for for exploratory analysis.

*Entropy is a little slower to compute up front.*

## Classification Error is...

http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.685.4929&rep=rep1&type=pdf

https://towardsdatascience.com/a-guide-to-decision-trees-for-machine-learning-and-data-science-fe2607241956

https://datamining.bus.utk.edu/Documents/Decision-Trees-for-Predictive-Modeling-(Neville).pdf

https://bricaud.github.io/personal-blog/entropy-in-decision-trees/

https://bambielli.com/til/2017-10-29-gini-impurity/#

https://medium.com/deep-math-machine-learning-ai/chapter-4-decision-trees-algorithms-b93975f7a1f1

notebook:

https://github.com/rasbt/python-machine-learning-book/blob/master/faq/decision-tree-binary.md